In [ ]:
import pandas as pd

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import scipy

In [ ]:
marker_to_behavior = {
    "chr1:42445143": ['2','4', '5', '6', '8', '11', '13', '108', '228',  "factor_2"],
    "chr34:18730331": ['41', '107', '109', 'factor_8'],
    "chr10:8354804": ['41', '107', '109', 'factor_8'],
    "chr10:8415970": ['41', '107', '109', 'factor_8'],
    "chr10:8752420": ['41', '107', '109', 'factor_8'],
    "chr10:8808920": ['41', '107', '109', 'factor_8'],
    "chr18:20591145": ['11', '42', '46', '167', '170', '171', '175'],
    "chr10:2790081":['11', '42', '46', '167', '170', '171', '175']
}
marker_to_morphology = {
    "chr1:42445143": ['246'],
    "chr34:18730331": ['121'],
    "chr10:8354804": ['125', '121'],
    "chr10:8415970": ['125', '121'],
    "chr10:8752420": ['125', '121'],
    "chr10:8808920": ['125', '121'],
    "chr18:20591145": ['121', '246'],
    "chr10:2790081":['121']
}

In [ ]:
height_file="/seq/vgb/vsohrab/darwins_dogs/pnas_behavior_morphology_gwas/input_files/DarwinsArk_AgeComposed_Height_N-3277.tsv"
neuter_file="/seq/vgb/vsohrab/darwins_dogs/pnas_behavior_morphology_gwas/input_files/DarwinsArk_Sex_NeuterStatus_N-3277.tsv"
pheno_file="/seq/vgb/bryc/data/darwins_dogs/2024-09-23_cache/2024-10-15_merged_phenotype_table.csv"
geno_file="/seq/vgb/bryc/data/darwins_dogs/pnas_dog_behavior_paper/target_sites_genotypes.csv"

In [ ]:
ph = pd.read_csv(pheno_file)
genos = pd.read_csv(geno_file)
height = pd.read_csv(height_file, sep = "\t",header=None, names = ["fam_id", "dog", "age", "height"])
neuter = pd.read_csv(neuter_file, sep = "\t", header=None, names = ["fam_id", "dog", "sex", "neuter"])

In [ ]:
ph.columns[:20]

In [ ]:
merge = ph.merge(genos, left_on='id', right_on='dog')
merge2 = merge.merge(height, left_on='id', right_on="dog")

In [ ]:
merge2

In [ ]:
sns.violinplot(data=merge2, x="246", y="chr18:20591145")

In [ ]:
merge2.hist(column=["chr34:18730331"], by='121')

In [ ]:
merge2['sex'] = merge2.sex.replace({'male': 0, 'female':1})
merge2['Q121'] = merge2['121']

In [ ]:
for snp in marker_to_morphology.keys():    
    for pheno in marker_to_morphology[snp]:
        # OLS is _very_ finicky about column names so we will copy ours over to "clean" names here
        merge2['test_snp'] = merge2[snp]
        merge2['phenotype'] = merge2[pheno]

        full_model = smf.ols(formula='phenotype ~ height + age_y + C(sex) + C(neutered) + test_snp', data=merge2)
        full_res = full_model.fit()
        #print(full_res.summary())
        full_ll = full_res.llf

        reduced_model = smf.ols(formula='phenotype ~ height + age_y + C(sex) + C(neutered)', data=merge2)
        reduced_res = reduced_model.fit()
        #print(reduced_res.summary())
        reduced_ll = reduced_res.llf

        #calculate likelihood ratio Chi-Squared test statistic
        LR_statistic = -2*(reduced_ll-full_ll)

        #calculate p-value of test statistic using degrees of freedom
        p_val = scipy.stats.chi2.sf(LR_statistic, 1)

        # If p_value < 0.05 then reject null
        if p_val < 0.05:
            print(f"*{snp} has a significant effect on phenotype {pheno}, p-value: {p_val}")
        else:
            print(f"{snp} does not have a significant effect on phenotype {pheno}, p-value: {p_val}")

In [ ]:
merge2.columns[30:100]

In [ ]:
## Test for correlation between SNPs and breed ancestries
ancestries = list(ph.columns[19:131])
anc = ancestries[0]
snp = "chr10:8752420"
col_list = ancestries +  list(marker_to_morphology.keys())

#print(col_list)
snp_anc_df = merge2[['Afghan Hound', 'Airedale Terrier', 'Akita', 'Alaskan Malamute', 'Alaskan Sled Dog', 'American Cocker Spaniel', 'American Pit Bull Terrier', 'Australian Cattle Dog', 'Australian Shepherd', 'Basenji', 'Basset Hound', 'Beagle', 'Bearded Collie', 'Belgian Malinois', 'Belgian Sheepdog', 'Belgian Tervuren', 'Bernese Mountain Dog', 'Bichon Frise', 'Bloodhound', 'Border Collie', 'Border Terrier', 'Borzoi', 'Boston Terrier', 'Bouvier Des Flanders', 'Boxer', 'Brittany', 'Bull Terrier', 'Bullmastiff', 'Cairn Terrier', 'Cavalier King Charles Spaniel', 'Chesapeake Bay Retriever', 'Chihuahua', 'Chinese Crested', 'Chinook', 'Chow Chow', 'Collie', 'Dachshund', 'Dalmatian', 'Doberman Pinscher', 'English Bulldog', 'English Cocker Spaniel', 'English Setter', 'English Shepherd', 'English Springer Spaniel', 'Entlebucher Mountain Dog', 'Finnish Spitz', 'French Bulldog', 'German Shepherd Dog', 'German Shorthaired Pointer', 'Golden Retriever', 'Gordon Setter', 'Great Dane', 'Great Pyrenees', 'Greater Swiss Mountain Dog', 'Greenland Sled Dog', 'Greyhound', 'Havanese', 'Irish Setter', 'Irish Wolfhound', 'Italian Greyhound', 'Jack Russell Terrier', 'Labrador Retriever', 'Lagotti Romagnoli', 'Leonberger', 'Lhasa Apso', 'Maltese', 'Mastiff', 'Miniature Pinscher', 'Miniature Schnauzer', 'Newfoundland', 'Norfolk Terrier', 'Norwegian Elkhound', 'Norwich Terrier', 'Nova Scotia Duck Tolling Retriever', 'Old English Sheepdog', 'Papillon', 'Pekingese', 'Pembroke Welsh Corgi', 'Pomeranian', 'Poodle', 'Portuguese Water Dog', 'Pug', 'Rhodesian Ridgeback', 'Rottweiler', 'Saint Bernard', 'Saluki', 'Samoyed', 'Schipperke', 'Scottish Terrier', 'Shar Pei', 'Shetland Sheepdog', 'Shiba Inu', 'Shih Tzu', 'Siberian Husky', 'Sloughi', 'Soft Coated Wheaten Terrier', 'Staffordshire Bull Terrier', 'Standard Schnauzer', 'Tibetan Mastiff', 'Tibetan Spaniel', 'Tibetan Terrier', 'Toy Poodle', 'Village Dog (China)', 'Village Dog (Nigeria)', 'Village Dog (Vietnam)', 'Vizsla', 'Weimaraner', 'West Highland White Terrier', 'Whippet', 'Wire Fox Terrier', 'Wirehaired Pointing Griffon', 'Yorkshire Terrier', 'chr1:42445143', 'chr34:18730331', 'chr10:8354804', 'chr10:8415970', 'chr10:8752420', 'chr10:8808920', 'chr18:20591145', 'chr10:2790081']]

In [ ]:
corr = snp_anc_df.corr()

In [ ]:
snp_corrs = corr.loc[['Afghan Hound', 'Airedale Terrier', 'Akita', 'Alaskan Malamute',  'American Cocker Spaniel', 'American Pit Bull Terrier', 'Australian Cattle Dog', 'Australian Shepherd', 'Basenji', 'Basset Hound', 'Beagle', 'Bearded Collie', 'Belgian Malinois',  'Belgian Tervuren', 'Bernese Mountain Dog', 'Bichon Frise', 'Bloodhound', 'Border Collie', 'Border Terrier', 'Borzoi', 'Boston Terrier', 'Boxer', 'Brittany', 'Bull Terrier', 'Bullmastiff', 'Cairn Terrier', 'Cavalier King Charles Spaniel', 'Chesapeake Bay Retriever', 'Chihuahua', 'Chinese Crested', 'Chinook', 'Chow Chow', 'Collie', 'Dachshund', 'Dalmatian', 'Doberman Pinscher', 'English Bulldog', 'English Cocker Spaniel', 'English Setter', 'English Shepherd', 'English Springer Spaniel', 'Entlebucher Mountain Dog', 'Finnish Spitz', 'French Bulldog', 'German Shepherd Dog', 'German Shorthaired Pointer', 'Golden Retriever', 'Gordon Setter', 'Great Dane', 'Great Pyrenees', 'Greater Swiss Mountain Dog', 'Greyhound', 'Havanese', 'Irish Setter', 'Irish Wolfhound', 'Italian Greyhound', 'Jack Russell Terrier', 'Labrador Retriever',  'Leonberger', 'Lhasa Apso', 'Maltese', 'Mastiff', 'Miniature Pinscher', 'Miniature Schnauzer', 'Newfoundland', 'Norfolk Terrier', 'Norwegian Elkhound',  'Nova Scotia Duck Tolling Retriever', 'Old English Sheepdog', 'Papillon', 'Pekingese', 'Pembroke Welsh Corgi', 'Pomeranian', 'Poodle', 'Portuguese Water Dog', 'Pug', 'Rhodesian Ridgeback', 'Rottweiler', 'Saint Bernard', 'Saluki', 'Samoyed', 'Schipperke', 'Scottish Terrier', 'Shar Pei', 'Shetland Sheepdog', 'Shiba Inu', 'Shih Tzu', 'Siberian Husky',  'Soft Coated Wheaten Terrier', 'Staffordshire Bull Terrier',  'Tibetan Mastiff', 'Tibetan Spaniel', 'Tibetan Terrier', 'Toy Poodle',  'Vizsla', 'Weimaraner', 'West Highland White Terrier', 'Whippet', 'Wire Fox Terrier', 'Wirehaired Pointing Griffon', 'Yorkshire Terrier'],['chr1:42445143', 'chr34:18730331', 'chr10:8354804', 'chr10:8415970', 'chr10:8752420', 'chr10:8808920', 'chr18:20591145', 'chr10:2790081']]
snp_corrs.style.background_gradient(cmap='coolwarm', axis=None)

In [ ]:
plt.figure(figsize=(13,30))
sns.heatmap(snp_corrs, annot = True, cmap='coolwarm')
plt.title("Correlation matrix of SNPs with dog breed estimates")
plt.xlabel("Associated SNPS")
plt.ylabel("Dog breeds")
plt.savefig("snp_correlation_with_breed.png", dpi=300)